In [98]:
import requests
import plotly
import plotly.express as px
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import time
import datetime
import json
from nltk import sent_tokenize
import nltk
import re
from tqdm import tqdm




In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

r=requests.get('https://www.aljazeera.com/where/mozambique/')
r.encoding = 'utf-8'
html = r.text

In [ ]:
print(html)

In [ ]:
# Creating a BeautifulSoup object from the HTML
soup = BeautifulSoup(html)
# Getting the text out of the soup
text = soup.get_text()

In [ ]:
for y in soup.findAll(class_="gc__content"):
  print(y.find(attrs={"aria-hidden":"true"}).text[:])

23 May 2022
18 Mar 2022
4 Mar 2022
1 Mar 2022
27 Jan 2022
12 Jan 2022
play
24 Sep 2021
8 Aug 2021
10 Jul 2021
23 Jun 2021
20 Jun 2021
9 Jun 2021
14 May 2021


In [ ]:
class article:
  def __init__(self, title, date, link, content):
    self.date = date
    self.link = link
    self.title = title
    self.content = content

In [102]:
articles = []
counter = 0
for y in tqdm(soup.findAll(class_="gc__content")):
  date_time0 = y.find(attrs={"aria-hidden":"true"}).text
  if(not date_time0.isalpha()):
    # print(y.span.text)
    # print(y.find(attrs={"aria-hidden":"true"}).text)
    # date_time = datetime.datetime.strptime(date_time0,"%d %b %Y")
    r1=requests.get('https://www.aljazeera.com/'+y.a["href"])
    r1.encoding = 'utf-8'
    html1 = r1.text
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"     
    
    # Creating a BeautifulSoup object from the HTML
    soup1 = BeautifulSoup(html1)
    # Getting the text out of the soup
    text1 = soup1.get_text()
    #print(text1)
    str_content = []
    #removing "follow alzeera paragraph"
    soup1.find('p',class_='site-footer__social-section-title css-0').decompose()
    for v in soup1.findAll('p'):
      #print(v.text)
      k = v.text.strip().strip('\n').replace('"','').replace("'",'')

      fg = k.split('\n')
      # print("watch ---->"+str(fg))
      if k == "":
        continue
      url = re.findall(regex,k) 
      if len(url)>0:
        continue
      h = sent_tokenize(k)
      for v in h:
        str_content.extend(v.strip().strip('\n').split('\n'))
      # print("new -------------------------> "+str(h))
      # print("new -------------------------> "+str(len(sent_tokenize(k))))
      # str_content.append(k)

    # print(str_content)
    curr = article(y.span.text, y.find(attrs={"aria-hidden":"true"}).text, y.a["href"], str_content)
    # print(y.find(attrs={"aria-hidden":"true"}).text)
    # print(date_time)
    # print(y.a["href"])
    articles.append(curr)
    counter+=1
    if counter == 10:
      break


    # split into sentences
# from nltk import sent_tokenize
# sentences = sent_tokenize(text)
# print(sentences[0])

 71%|███████▏  | 10/14 [00:00<00:00, 13.03it/s]


In [100]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump([z.__dict__ for z in articles], f, ensure_ascii=False, indent=4)

In [74]:

# Opening JSON file
f1 = open('data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f1)
f1.close()
print(data[0]['content'])

['The floods are happening only a month after torrential rains killed 435 people in the KwaZulu Natal province.', 'Heavy rains in South Africa have forced hundreds of people to flee their homes on the east coast, according to officials, only a month after the worst floods in living memory killed more than 400.', 'The floods over the weekend hit infrastructure, roads, bridges and damaged buildings, mainly in the port city of Durban, the capital of KwaZulu-Natal province and home to some 3.9 million people.', '“Some areas are inaccessible and have become islands at this stage,” KwaZulu-Natal Premier Sihle Zikalala said during a news conference on Sunday.', 'No fatalities were immediately recorded, but Zikalala added: “We are yet to receive a full comprehensive report on the impact of these latest heavy rains, but as reports come in a worrying picture is however emerging.”', 'In April, torrential rain led to floods, landslides and the death of 435 people in KwaZulu-Natal and authorities s

In [109]:
from textblob import TextBlob
mapping = {}
for i in tqdm(data):
  textblob_sentiment=[]
  for s in i['content']:
    txt= TextBlob(s)
    a= txt.sentiment.polarity
    b= txt.sentiment.subjectivity
    print(b)
    textblob_sentiment.append([s,a,b])
  # print(textblob_sentiment)
  mapping[i['title']] = textblob_sentiment

 40%|████      | 4/10 [00:00<00:00, 22.68it/s]

0.5
0.5333333333333333
0.3333333333333333
0.0
0.65
0.2
0.1
0.4
0.3
0.8125
0.45
0.0
0.0
0.0
0.0
0.5
0.0
0.875
0.4166666666666667
0.4125
0.125
0.0
0.35
0.45
0.13333333333333333
0.4772727272727273
0.4323863636363636
0.0
0.47500000000000003
0.5
0.0
0.0
0.0
0.38
0.2
0.0
1.0
0.33888888888888885
0.6517857142857143
0.125
0.22575757575757574
0.0
0.7
1.0
0.4
0.0
0.35000000000000003
0.3333333333333333
0.32
0.0
0.4000000000000001
0.0
0.28055555555555556
0.5625
0.4
0.375
0.43833333333333335
0.0
0.4
0.25
0.45
0.0
0.37744107744107747
0.25
0.0
0.4375
0.4
0.26666666666666666
0.3020833333333333
0.5375
0.0
0.0
1.0
0.875
0.0
0.2
0.31666666666666665
0.0
0.55
0.2
0.0
0.0
0.5


100%|██████████| 10/10 [00:00<00:00, 24.81it/s]

0.03333333333333333
0.0
0.0
0.8333333333333334
0.25
0.0
0.4
0.4
0.4
0.0
0.0
0.4625
0.06666666666666667
0.0
0.5
0.2833333333333333
0.0
0.875
0.32499999999999996
0.5
0.0
0.0
0.5
0.3333333333333333
0.6
0.0
0.17500000000000002
0.2633333333333333
0.1111111111111111
0.19999999999999998
0.0
0.0
0.0
0.6
0.8333333333333334
0.75
0.5
1.0
0.0
0.4
0.0625
0.05
0.325
0.8888888888888888
0.3194444444444444
0.5714285714285714
0.35
0.0
0.2
0.4
0.3
0.5111111111111111
0.35624999999999996
0.5
0.0
0.08333333333333333
0.9
0.0
0.2
0.3354166666666667
0.27999999999999997
0.0
0.4
0.5333333333333333
0.5
1.0
0.13333333333333333
0.19999999999999998
0.18888888888888888
0.24000000000000005
0.0
0.0
0.10555555555555556
0.0
0.6
0.0
0.5022727272727273
0.6
0.37142857142857144
0.35
0.4
0.17500000000000002
0.0
0.5
0.2
0.5375
0.40625
0.4
1.0
0.15625
0.19166666666666665
0.3333333333333333
0.1
0.625
0.0
0.2333333333333333
0.0625
0.17500000000000002
0.0
0.5
0.2
0.75
0.40625
0.45729166666666665
0.0
0.41666666666666663
0.520833333

In [110]:
df_textblob = {}
for key in mapping:
  df_textblob[key] = pd.DataFrame(mapping[key], columns =['Sentence', 'Polarity', 'Subjectivity'])

In [105]:
means={}
for key in tqdm(mapping):
  means[key] = df_textblob[key]['Polarity'].mean()
  # print(key,df_textblob[key]['Polarity'].mean())

100%|██████████| 10/10 [00:00<00:00, 2593.24it/s]


In [115]:
means_subjectivity={}
for key in tqdm(mapping):
  means_subjectivity[key] = df_textblob[key]['Subjectivity'].mean()
  # print(key,df_textblob[key]['Polarity'].mean())

100%|██████████| 10/10 [00:00<00:00, 5356.03it/s]


In [ ]:
for h in means:
  print(h,means[h])

Floods hit South Africa’s KwaZu­lu-Na­tal province again 0.08186274509803922
Mozam­bique: Cy­clone Gombe death toll ris­es to 53 -0.14089285714285713
Mozam­bique an­nounces new prime min­is­ter af­ter cab­i­net reshuf­fle 0.07295454545454545
Analy­sis: Can African gas re­place Russ­ian sup­plies to Eu­rope? 0.05373663887552777
Dozens dead from Trop­i­cal Storm Ana in south­ern Africa 0.012499999999999994
South­ern Africa bloc SADC ex­tends Mozam­bique mis­sion 0.11180555555555555
In Mozam­bique, Kagame says Rwan­dan troops’ work not over 0.08444734804490904
Rwan­da, Mozam­bique forces re­cap­ture port city from rebels 0.13369791666666667
Rwan­da de­ploys 1,000 sol­diers to Mozam­bique’s Cabo Del­ga­do 0.0008942099567099578


In [ ]:
# import seaborn as sns
# sns.displot(df_textblob['Floods hit South Africa’s KwaZu­lu-Na­tal province again']["Subjectivity"], height= 5, aspect=1.8)
# plt.xlabel("Sentence Subjectivity (Textblob)")

In [ ]:
# sns.displot(df_textblob['Floods hit South Africa’s KwaZu­lu-Na­tal province again']["Polarity"], height= 5, aspect=1.8)
# plt.xlabel("Sentence Polarity (Textblob)")
# plt.title('Floods hit South Africa’s KwaZu­lu-Na­tal province again')

In [ ]:
# for key in df_textblob:
#   sns.displot(df_textblob[key]["Subjectivity"], height= 5, aspect=1.8)
#   plt.xlabel("Sentence Subjectivity (Textblob)")
#   plt.title(key+"  | polarity = "+str(means[key]))

In [ ]:
# for key in df_textblob:
#   sns.displot(df_textblob[key]["Polarity"], height= 5, aspect=1.8)
#   plt.xlabel("Sentence Polarity (Textblob)")
#   plt.title(key+"  | polarity = "+str(means[key]))

In [81]:
def classify_sentiment(value1):
  if value1>=0.1:
    return "Positive"
  elif value1<0:
    return "Negative"
  else :
    return "Neutral"

In [114]:
for key in tqdm(df_textblob):
  # print(key)
  fig = px.histogram(df_textblob[key], x='Polarity', histnorm='percent', title=key+"  | polarity = "+str(means[key])+" "+classify_sentiment(means[key]))
  #print(means[key])
  fig.show()
  

  0%|          | 0/10 [00:00<?, ?it/s]

 20%|██        | 2/10 [00:00<00:00, 14.70it/s]

 40%|████      | 4/10 [00:00<00:00, 14.82it/s]

 60%|██████    | 6/10 [00:00<00:00, 14.90it/s]

 80%|████████  | 8/10 [00:00<00:00, 15.27it/s]

100%|██████████| 10/10 [00:00<00:00, 14.96it/s]


In [116]:

for key in tqdm(df_textblob):
  print(key)
  fig = px.histogram(df_textblob[key], x='Subjectivity', histnorm='percent', title=key+"  | Subjectivity = "+str(means_subjectivity[key]))
  fig.show()

  0%|          | 0/10 [00:00<?, ?it/s]

Floods hit South Africa’s KwaZu­lu-Na­tal province again


Mozam­bique: Cy­clone Gombe death toll ris­es to 53


 20%|██        | 2/10 [00:00<00:00, 16.54it/s]

Mozam­bique an­nounces new prime min­is­ter af­ter cab­i­net reshuf­fle


...

Rwan­da, Mozam­bique forces re­cap­ture port city from rebels


 80%|████████  | 8/10 [00:00<00:00, 14.44it/s]

Rwan­da de­ploys 1,000 sol­diers to Mozam­bique’s Cabo Del­ga­do


South­ern African na­tions agree to de­ploy forces to Mozam­bique


100%|██████████| 10/10 [00:00<00:00, 14.64it/s]
